<a href="https://colab.research.google.com/github/yzhan480/CS535_Final_Project/blob/main/CS535Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Team: No. 48
# Member: Yao Zhang(B00961692) & Xu Yang(B00924331)
# Email: yzhan480@binghamton.edu & xyang91@binghamton.edu

Notice: if run all my code, the below cell may need to be revised

In [3]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import os
import time
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

Notice: The path should be your right file path.

In [5]:
training_data = pd.read_pickle(r'/content/drive/My Drive/training_set.pkl')

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Task 1: Adding label

In [7]:
label = []
for stock in range(2000):
  label.append([])

I think in each stock row 0 to 2200 has a label respectively except row 2201.

In [8]:
for stock in range(2000):
  a = 0
  b = 0
  c = 0
  percent_change = []
  for row in range(2201):
    percentage_change = (training_data[stock]['Close'][row+1]-training_data[stock]['Close'][row])/(training_data[stock]['Close'][row])
    percent_change.append(percentage_change)
  
  percent_change_origin = percent_change.copy()
  percent_change.sort()

  
  for row in range(2201):
    if percent_change_origin[row] < percent_change[733]:
      label[stock].append([1,0,0])
      a += 1
    if percent_change_origin[row] >= percent_change[733] and percent_change_origin[row] < percent_change[1467]:
      label[stock].append([0,1,0])
      b += 1
    if percent_change_origin[row] >= percent_change[1467]:
      label[stock].append([0,0,1])
      c += 1

a,b,c # for verification

(733, 734, 734)

Here, lable is a list. And each element of the label is also a list which contains the labels of corresponding stock. So len(label) is 2000, len(label[i]) is 2201. And label[i][j] is a label for ith stock jth day because day 2201 does not have a label. And one label is like (0,1,0) one-hot vector which represents this day belongs to second class "no big change". And first class is "decrease", third class is "increase".

In [9]:
label = np.array(label)
type(label), len(label[0])

(numpy.ndarray, 2201)

## Task 2: Feature Engineering

Add features. Some features are linear dependent but I still add them because in practice, they still can influence the training results. Here features are like hyperparameters. I tune a lot of times.

In [10]:
for stock in range(2000):
  training_data[stock] = training_data[stock].assign(F1 = training_data[stock].Close *1.1 )
  training_data[stock] = training_data[stock].assign(F2 = training_data[stock].Close *1.2 )
  training_data[stock] = training_data[stock].assign(F3 = training_data[stock].Close *1.3 )
  training_data[stock] = training_data[stock].assign(F4 = training_data[stock].Close *1.4 )
  training_data[stock] = training_data[stock].assign(F5 = training_data[stock].Close *1.5 )
  training_data[stock] = training_data[stock].assign(F6 = training_data[stock].Close ** 2 )
  training_data[stock] = training_data[stock].assign(F7 = training_data[stock].Close ** 3 )
  training_data[stock] = training_data[stock].assign(F8 = training_data[stock].Close ** 4 * 1.5)
  training_data[stock] = training_data[stock].assign(F9 = np.sqrt(np.abs(training_data[stock].Close)))
  training_data[stock] = training_data[stock].assign(F10 = training_data[stock].Close * training_data[stock].Open)
  training_data[stock] = training_data[stock].assign(F11 = (training_data[stock].Close **2) * training_data[stock].Open)
  training_data[stock] = training_data[stock].assign(F12 = training_data[stock].Close * training_data[stock].High)
  training_data[stock] = training_data[stock].assign(F13 = training_data[stock].Close * training_data[stock].Low)
  training_data[stock] = training_data[stock].assign(F14 = training_data[stock].Close * training_data[stock].Volume)
  training_data[stock] = training_data[stock].assign(F15 = training_data[stock].Close * training_data[stock].Open**2)
  training_data[stock] = training_data[stock].assign(F16 = training_data[stock].Close * training_data[stock].High**2)
  training_data[stock] = training_data[stock].assign(F17 = training_data[stock].Close * training_data[stock].Low**2)
  training_data[stock] = training_data[stock].assign(F18 = training_data[stock].Close * training_data[stock].Volume**2)
  training_data[stock] = training_data[stock].assign(F19 = (training_data[stock].Close - training_data[stock].Open)/training_data[stock].Open)
  training_data[stock] = training_data[stock].assign(F20 = (training_data[stock].Open - training_data[stock].Close)/training_data[stock].Close)
  training_data[stock] = training_data[stock].assign(F21 = (training_data[stock].High - training_data[stock].Low)/training_data[stock].Low)
  training_data[stock] = training_data[stock].assign(F22 = (training_data[stock].Low - training_data[stock].High)/training_data[stock].High)
  
  percent_change = []
  for row in range(2201):
    percentage_change = (training_data[stock]['Close'][row+1]-training_data[stock]['Close'][row])/(training_data[stock]['Close'][row])
    percent_change.append(percentage_change)  
  percent_change.append(0.001) #initialize for row 2201
  percent_change = np.array(percent_change)
  
  training_data[stock] = training_data[stock].assign(F23 = percent_change)
  training_data[stock] = training_data[stock].assign(F24 = percent_change*10)
  training_data[stock] = training_data[stock].assign(F25 = percent_change*100)
  training_data[stock] = training_data[stock].assign(F26 = (percent_change**2) * 100)
  training_data[stock] = training_data[stock].assign(F27 = (percent_change**2) * 1000)
  training_data[stock] = training_data[stock].assign(F28 = (percent_change**2) * 10000)
  training_data[stock] = training_data[stock].assign(F29 = (percent_change**3) * 100000)
  training_data[stock] = training_data[stock].assign(F30 = 10*(training_data[stock].Close - training_data[stock].Open)/(training_data[stock].Open))
  training_data[stock] = training_data[stock].assign(F31 = 100*(training_data[stock].Close - training_data[stock].Open)/(training_data[stock].Open))
  training_data[stock] = training_data[stock].assign(F32 = 10*(training_data[stock].Open - training_data[stock].Close)/(training_data[stock].Close))
  training_data[stock] = training_data[stock].assign(F33 = 100*(training_data[stock].Open - training_data[stock].Close)/(training_data[stock].Close))
  training_data[stock] = training_data[stock].assign(F34 = 10*(training_data[stock].High - training_data[stock].Low)/(training_data[stock].Low))
  training_data[stock] = training_data[stock].assign(F35 = 100*(training_data[stock].High - training_data[stock].Low)/(training_data[stock].Low))
  training_data[stock] = training_data[stock].assign(F36 = 10*(training_data[stock].Low - training_data[stock].High)/(training_data[stock].High))
  training_data[stock] = training_data[stock].assign(F37 = 100*(training_data[stock].Low - training_data[stock].High)/(training_data[stock].High))
  training_data[stock] = training_data[stock].assign(F38 = training_data[stock].Close ** 4 * 5)
  training_data[stock] = training_data[stock].assign(F39 = training_data[stock].Close ** 2 * 1.2)
  training_data[stock] = training_data[stock].assign(F40 = training_data[stock].Close ** 2 * 1.1)
  training_data[stock] = training_data[stock].assign(F41 = np.random.rand(2202)/100)
  training_data[stock] = training_data[stock].assign(F42 = np.sqrt(np.abs(percent_change)))
  training_data[stock] = training_data[stock].assign(F43 = np.sqrt(np.abs(percent_change))*10)
  training_data[stock] = training_data[stock].assign(F44 = np.sqrt(np.abs(percent_change))*20)
  training_data[stock] = training_data[stock].assign(F45 = np.sqrt(np.abs(percent_change))*30)
  training_data[stock] = training_data[stock].assign(F46 = np.sqrt(np.abs(percent_change))*40)
  training_data[stock] = training_data[stock].assign(F47 = percent_change*10 + np.random.rand(2202)/100)
  training_data[stock] = training_data[stock].assign(F48 = percent_change*100 + np.random.rand(2202)/10)
  training_data[stock] = training_data[stock].assign(F49 = percent_change*100 + np.random.rand(2202)/100)
  training_data[stock] = training_data[stock].assign(F50 = np.ones(2202)/1000)


In [11]:
for stock in range(2000):
  training_data[stock] = training_data[stock].assign(F51 = 0.9 * training_data[stock].Close *1.1 )
  training_data[stock] = training_data[stock].assign(F52 = 0.9 * training_data[stock].Close *1.2 )
  training_data[stock] = training_data[stock].assign(F53 = 0.9 * training_data[stock].Close *1.3 )
  training_data[stock] = training_data[stock].assign(F54 = 0.9 * training_data[stock].Close *1.4 )
  training_data[stock] = training_data[stock].assign(F55 = 0.9 * training_data[stock].Close *1.5 )
  training_data[stock] = training_data[stock].assign(F56 = 0.9 * training_data[stock].Close ** 2 )
  training_data[stock] = training_data[stock].assign(F57 = 0.9 * training_data[stock].Close ** 3 )
  training_data[stock] = training_data[stock].assign(F58 = 0.9 * training_data[stock].Close ** 4 * 1.5)
  training_data[stock] = training_data[stock].assign(F59 = 0.9 * np.sqrt(np.abs(training_data[stock].Close)))
  training_data[stock] = training_data[stock].assign(F60 = 0.9 * training_data[stock].Close * training_data[stock].Open)
  training_data[stock] = training_data[stock].assign(F61 = 0.9 * (training_data[stock].Close **2) * training_data[stock].Open)
  training_data[stock] = training_data[stock].assign(F62 = 0.9 * training_data[stock].Close * training_data[stock].High)
  training_data[stock] = training_data[stock].assign(F63 = 0.9 * training_data[stock].Close * training_data[stock].Low)
  training_data[stock] = training_data[stock].assign(F64 = 0.9 * training_data[stock].Close * training_data[stock].Volume)
  training_data[stock] = training_data[stock].assign(F65 = 0.9 * training_data[stock].Close * training_data[stock].Open**2)
  training_data[stock] = training_data[stock].assign(F66 = 0.9 * training_data[stock].Close * training_data[stock].High**2)
  training_data[stock] = training_data[stock].assign(F67 = 0.9 * training_data[stock].Close * training_data[stock].Low**2)
  training_data[stock] = training_data[stock].assign(F68 = 0.9 * training_data[stock].Close * training_data[stock].Volume**2)
  training_data[stock] = training_data[stock].assign(F69 = 0.9 * (training_data[stock].Close - training_data[stock].Open)/training_data[stock].Open)
  training_data[stock] = training_data[stock].assign(F70 = 0.9 * (training_data[stock].Open - training_data[stock].Close)/training_data[stock].Close)
  training_data[stock] = training_data[stock].assign(F71 = 0.9 * (training_data[stock].High - training_data[stock].Low)/training_data[stock].Low)
  training_data[stock] = training_data[stock].assign(F72 = 0.9 * (training_data[stock].Low - training_data[stock].High)/training_data[stock].High)
  
  percent_change = []
  for row in range(2201):
    percentage_change = (training_data[stock]['Close'][row+1]-training_data[stock]['Close'][row])/(training_data[stock]['Close'][row])
    percent_change.append(percentage_change)  
  percent_change.append(0.001) #initialize for row 2201
  percent_change = np.array(percent_change)
  
  training_data[stock] = training_data[stock].assign(F73 = 0.9 * percent_change)
  training_data[stock] = training_data[stock].assign(F74 = 0.9 * percent_change*10)
  training_data[stock] = training_data[stock].assign(F75 = 0.9 * percent_change*100)
  training_data[stock] = training_data[stock].assign(F76 = 0.9 * (percent_change**2) * 100)
  training_data[stock] = training_data[stock].assign(F77 = 0.9 * (percent_change**2) * 1000)
  training_data[stock] = training_data[stock].assign(F78 = 0.9 * (percent_change**2) * 10000)
  training_data[stock] = training_data[stock].assign(F79 = 0.9 * (percent_change**3) * 100000)
  training_data[stock] = training_data[stock].assign(F80 = 0.9 * 10*(training_data[stock].Close - training_data[stock].Open)/(training_data[stock].Open))
  training_data[stock] = training_data[stock].assign(F81 = 0.9 * 100*(training_data[stock].Close - training_data[stock].Open)/(training_data[stock].Open))
  training_data[stock] = training_data[stock].assign(F82 = 0.9 * 10*(training_data[stock].Open - training_data[stock].Close)/(training_data[stock].Close))
  training_data[stock] = training_data[stock].assign(F83 = 0.9 * 100*(training_data[stock].Open - training_data[stock].Close)/(training_data[stock].Close))
  training_data[stock] = training_data[stock].assign(F84 = 0.9 * 10*(training_data[stock].High - training_data[stock].Low)/(training_data[stock].Low))
  training_data[stock] = training_data[stock].assign(F85 = 0.9 * 100*(training_data[stock].High - training_data[stock].Low)/(training_data[stock].Low))
  training_data[stock] = training_data[stock].assign(F86 = 0.9 * 10*(training_data[stock].Low - training_data[stock].High)/(training_data[stock].High))
  training_data[stock] = training_data[stock].assign(F87 = 0.9 * 100*(training_data[stock].Low - training_data[stock].High)/(training_data[stock].High))
  training_data[stock] = training_data[stock].assign(F88 = 0.9 * training_data[stock].Close ** 4 * 5)
  training_data[stock] = training_data[stock].assign(F89 = 0.9 * training_data[stock].Close ** 2 * 1.2)
  training_data[stock] = training_data[stock].assign(F90 = 0.9 * training_data[stock].Close ** 2 * 1.1)
  training_data[stock] = training_data[stock].assign(F91 = 0.9 * np.random.rand(2202)/100)
  training_data[stock] = training_data[stock].assign(F92 = 0.9 * np.sqrt(np.abs(percent_change)))
  training_data[stock] = training_data[stock].assign(F93 = 0.9 * np.sqrt(np.abs(percent_change))*10)
  training_data[stock] = training_data[stock].assign(F94 = 0.9 * np.sqrt(np.abs(percent_change))*30)
  training_data[stock] = training_data[stock].assign(F95 = 0.9 * np.sqrt(np.abs(percent_change))*20)
  training_data[stock] = training_data[stock].assign(F96 = 0.9 * np.sqrt(np.abs(percent_change))*40)
  training_data[stock] = training_data[stock].assign(F97 = 0.9 * percent_change*10 + np.random.rand(2202)/100)
  training_data[stock] = training_data[stock].assign(F98 = 0.9 * percent_change*100 + np.random.rand(2202)/10)
  training_data[stock] = training_data[stock].assign(F99 = 0.9 * percent_change*100 + np.random.rand(2202)/100)
  training_data[stock] = training_data[stock].assign(F100 = 0.9 * np.ones(2202)/1000)

### Print the last 5 data points of each stock.

In [12]:
for stock in range(2000):
  print(training_data[stock].tail())

流式输出内容被截断，只能显示最后 5000 行内容。
2198  2.744256  1.829504  3.659008 -0.085364 -0.869433 -0.928752  0.0009  
2199  2.727736  1.818490  3.636981 -0.085245 -0.914217 -0.909187  0.0009  
2200  1.239998  0.826665  1.653331 -0.016711 -0.131704 -0.183761  0.0009  
2201  0.853815  0.569210  1.138420  0.016201  0.141119  0.092030  0.0009  

[5 rows x 105 columns]
          Open      High       Low     Close    Volume        F1        F2  \
2197  0.496450  0.500609  0.496450  0.497768  0.071710  0.547545  0.597322   
2198  0.497616  0.497616  0.489349  0.491479  0.036097  0.540627  0.589775   
2199  0.491783  0.493051  0.487016  0.490211  0.035099  0.539232  0.588253   
2200  0.490160  0.491773  0.487726  0.489450  0.070256  0.538395  0.587340   
2201  0.489450  0.491378  0.485697  0.487624  0.088028  0.536387  0.585149   

            F3        F4        F5  ...       F91       F92       F93  \
2197  0.647099  0.696876  0.746652  ...  0.004966  0.101165  1.011649   
2198  0.638923  0.688071  0.737219

## Task 3

In [13]:
label[0].shape

(2201, 3)

0-4 stocks as training data, 6th stock as validation data. And transforming these data to torch tensor costs about 30mins. So, reading 2000 stocks as training data is impossible for me. Prediction performance is bad on validation dataset but after feature selection I will try to improve the performance.

In [14]:
def prepare_training_data(training_data, label):
  for stock in range(5):
    for i in range(2201 - 100):
      df = training_data[stock][i:i+100]
      if stock == 0 and i == 0:
        X_train = torch.Tensor(df.values).reshape(1,100,105)
        y_train = torch.Tensor(label[stock][i+100]).reshape(1,3)
      else:
        X_train = torch.cat((X_train, torch.Tensor(df.values).reshape(1,100,105)),0)
        y_train = torch.cat((y_train, torch.Tensor(label[stock][i+100]).reshape(1,3)),0)
      
  
  for stock in range(5,6):
    for i in range(2201 - 100):
      df = training_data[stock][i:i+100]
      if stock == 5 and i == 0:
        X_valid = torch.Tensor(df.values).reshape(1,100,105)
        y_valid = torch.Tensor(label[stock][i+100]).reshape(1,3)
      else:
        X_valid = torch.cat((X_valid, torch.Tensor(df.values).reshape(1,100,105)),0)
        y_valid = torch.cat((y_valid, torch.Tensor(label[stock][i+100]).reshape(1,3)),0)        

  return X_train, y_train, X_valid, y_valid

In [15]:
X_train, y_train, X_valid, y_valid = prepare_training_data(training_data, label)

The shape of my four data

In [16]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

(torch.Size([10505, 100, 105]),
 torch.Size([10505, 3]),
 torch.Size([2101, 100, 105]),
 torch.Size([2101, 3]))

In [17]:
y_train = y_train.to(torch.float32)
y_valid = y_valid.to(torch.float32)

In [18]:
class Stocks_Data(Dataset):
  '''custom dataset'''
  def __init__(self, data_X, data_Y, transform=None):
    self.X = data_X.to(device)
    self.Y = data_Y.to(device)
    self.transform = transform

  def __len__(self):
    """Returns the size (the number of samples) of the dataset.
    """
    return len(self.Y)
  
  def __getitem__(self, i):
    """Returns the i-th sample and label and applies any transforms defined.

      Args:
        i (int): The index of sample in the data array to retrieve. 
    """  
    label = self.Y[i]
    sample = self.X[i]
    if self.transform:
        sample = self.transform(sample)

    return sample, label

In [19]:
class Model_1(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_1, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.BatchNorm1d(hidden_size_1),
        nn.ReLU(),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.BatchNorm1d(hidden_size_2),
        nn.ReLU(),
        nn.Linear(hidden_size_2, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Linear(256, 32),
        nn.BatchNorm1d(32),
        nn.ReLU(),
        nn.Linear(32, 3),
    )
  def forward(self, input):
    return self.main(input)

In [20]:
class Model_2(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_2, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.BatchNorm1d(hidden_size_1),
        nn.ReLU(),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.BatchNorm1d(hidden_size_2),
        nn.ReLU(),
        nn.Linear(hidden_size_2, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Linear(256, 32),
        nn.BatchNorm1d(32),
        nn.ReLU(),
        nn.Linear(32, 3),
        nn.Softmax(dim=1),
    )
  def forward(self, input):
    return self.main(input)

In [21]:
class Model_3(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3, prob = 0.2):
    super(Model_3, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.BatchNorm1d(hidden_size_1),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.BatchNorm1d(hidden_size_2),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(hidden_size_2, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(256, 32),
        nn.BatchNorm1d(32),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(32, 3),
    )
  def forward(self, input):
    return self.main(input)

In [22]:
class Model_4(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3, prob = 0.2):
    super(Model_4, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.LayerNorm(hidden_size_1),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.LayerNorm(hidden_size_2),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(hidden_size_2, 512),
        nn.LayerNorm(512),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(512, 256),
        nn.LayerNorm(256),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(256, 32),
        nn.LayerNorm(32),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(32, 3),
        nn.Softmax(dim=1),
    )
  def forward(self, input):
    return self.main(input)

In [23]:
class Model_5(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_5, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.LayerNorm(hidden_size_1),
        nn.ReLU(),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.LayerNorm(hidden_size_2),
        nn.ReLU(),
        nn.Linear(hidden_size_2, 512),
        nn.LayerNorm(512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.LayerNorm(256),
        nn.ReLU(),
        nn.Linear(256, 32),
        nn.LayerNorm(32),
        nn.ReLU(),
        nn.Linear(32, 3),
    )
  def forward(self, input):
    return self.main(input)

In [24]:
class Model_6(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3):
    super(Model_6, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.LayerNorm(hidden_size_1),
        nn.ReLU(),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.LayerNorm(hidden_size_2),
        nn.ReLU(),
        nn.Linear(hidden_size_2, 512),
        nn.LayerNorm(512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.LayerNorm(256),
        nn.ReLU(),
        nn.Linear(256, 32),
        nn.LayerNorm(32),
        nn.ReLU(),
        nn.Linear(32, 3),
        nn.Softmax(dim=1),
    )
  def forward(self, input):
    return self.main(input)

In [25]:
class Model_7(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3, prob = 0.2):
    super(Model_7, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.LayerNorm(hidden_size_1),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.LayerNorm(hidden_size_2),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(hidden_size_2, 512),
        nn.LayerNorm(512),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(512, 256),
        nn.LayerNorm(256),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(256, 32),
        nn.LayerNorm(32),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(32, 3),
    )
  def forward(self, input):
    return self.main(input)

In [26]:
class Model_8(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, num_class = 3, prob = 0.2):
    super(Model_8, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, hidden_size_1),
        nn.LayerNorm(hidden_size_1),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(hidden_size_1, hidden_size_2),
        nn.LayerNorm(hidden_size_2),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(hidden_size_2, 512),
        nn.LayerNorm(512),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(512, 256),
        nn.LayerNorm(256),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(256, 32),
        nn.LayerNorm(32),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(32, 3),
        nn.Softmax(dim=1),
    )
  def forward(self, input):
    return self.main(input)

In [27]:
class Model_9(nn.Module):
  def __init__(self, input_size, num_class = 3, prob = 0.5):
    super(Model_9, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, 512),
        nn.LayerNorm(512),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(512, 256),
        nn.LayerNorm(256),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(256, 3),
    )
  def forward(self, input):
    return self.main(input)

In [28]:
class Model_X(nn.Module):
  def __init__(self, input_size, num_class = 3, prob = 0.5):
    super(Model_X, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(256, 3),
    )
  def forward(self, input):
    return self.main(input)

In [29]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):   
        # Compute prediction and loss
        
        pred = model(X).to(torch.float32)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    true_positive, pred_positive = 0, 0

    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            
            pred = model(X)
            
            test_loss += loss_fn(pred, y).item()
            #correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
            for i in range(len(y)):
                
                if pred[i].argmax().type(torch.float).item()>1:
                    pred_positive += 1
                    
                if pred[i].argmax().type(torch.float).item() == y[i].argmax().type(torch.float).item():
                    correct += 1
                    if pred[i].argmax().type(torch.float).item()>1:
                        true_positive += 1
                        

    percentage_of_positive_predictions = pred_positive / size
    precision = true_positive / pred_positive
    test_loss /= num_batches
    correct /= size
    print(f"Validation: \n Accuracy: {(100*correct):>0.1f}%, Precision: {(100*precision):>0.1f}%, \
Percentage of positive predictions: {(100*percentage_of_positive_predictions):>0.1f}%, Avg loss: {test_loss:>8f}")

In [30]:
training_dataset = Stocks_Data(X_train, y_train)
valid_dataset = Stocks_Data(X_valid, y_valid)

In [31]:
train_dataloader = DataLoader(training_dataset, batch_size=200)
valid_dataloader = DataLoader(valid_dataset, batch_size=200)

If you meet error "divided by zero", then increase the training epochs and after more epochs start the test_loop. (For TA, if you run my code and meet such problem)

In [32]:
model_1 = Model_1(input_size = 10500, hidden_size_1 = 12500, hidden_size_2 = 6400).to(device)
learning_rate = 1e-4
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_1.parameters(), lr=learning_rate)

epochs = 200
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_1, loss_fn, optimizer)
    if t > 90: # here 
        test_loop(valid_dataloader, model_1, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, model_1, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.731618  [    0/10505]
Epoch 2
-------------------------------
loss: 0.682039  [    0/10505]
Epoch 3
-------------------------------
loss: 0.630293  [    0/10505]
Epoch 4
-------------------------------
loss: 0.565467  [    0/10505]
Epoch 5
-------------------------------
loss: 0.501316  [    0/10505]
Epoch 6
-------------------------------
loss: 0.456510  [    0/10505]
Epoch 7
-------------------------------
loss: 0.426149  [    0/10505]
Epoch 8
-------------------------------
loss: 0.400021  [    0/10505]
Epoch 9
-------------------------------
loss: 0.368468  [    0/10505]
Epoch 10
-------------------------------
loss: 0.343760  [    0/10505]
Epoch 11
-------------------------------
loss: 0.330054  [    0/10505]
Epoch 12
-------------------------------
loss: 0.283230  [    0/10505]
Epoch 13
-------------------------------
loss: 0.270283  [    0/10505]
Epoch 14
-------------------------------
loss: 0.244507  [    0/10505]
Epoch 15
------

In [33]:
model_X = Model_X(input_size = 10500, prob = 0.1).to(device)
learning_rate = 1e-4
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_X.parameters(), lr=learning_rate)

epochs = 200
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_X, loss_fn, optimizer)
    if t > 90:
        test_loop(valid_dataloader, model_X, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, model_X, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.674144  [    0/10505]
Epoch 2
-------------------------------
loss: 0.560937  [    0/10505]
Epoch 3
-------------------------------
loss: 0.545942  [    0/10505]
Epoch 4
-------------------------------
loss: 0.517360  [    0/10505]
Epoch 5
-------------------------------
loss: 0.491878  [    0/10505]
Epoch 6
-------------------------------
loss: 0.461452  [    0/10505]
Epoch 7
-------------------------------
loss: 0.414214  [    0/10505]
Epoch 8
-------------------------------
loss: 0.367807  [    0/10505]
Epoch 9
-------------------------------
loss: 0.336127  [    0/10505]
Epoch 10
-------------------------------
loss: 0.296758  [    0/10505]
Epoch 11
-------------------------------
loss: 0.245366  [    0/10505]
Epoch 12
-------------------------------
loss: 0.226481  [    0/10505]
Epoch 13
-------------------------------
loss: 0.202487  [    0/10505]
Epoch 14
-------------------------------
loss: 0.170063  [    0/10505]
Epoch 15
------

### After run 10 models, I find that the performance is bad and is always around 36% so I delete 8 models to save running time. I'll try to improve the performance in next two tasks.

## Task 4

Correlation Coefficient

In [34]:
cor = training_data[200].corr()
cor

,Open,High,Low,Close,Volume,F1,F2,F3,F4,F5,...,F91,F92,F93,F94,F95,F96,F97,F98,F99,F100
Open,1.000000,0.999047,0.999021,0.998199,-0.114669,0.998199,0.998199,0.998199,0.998199,0.998199,...,-0.006196,-0.046304,-0.046304,-0.046304,-0.046304,-0.046304,-0.036349,-0.039283,-0.037199,NaN
High,0.999047,1.000000,0.998460,0.998978,-0.102897,0.998978,0.998978,0.998978,0.998978,0.998978,...,-0.006468,-0.044692,-0.044692,-0.044692,-0.044692,-0.044692,-0.036955,-0.039891,-0.037827,NaN
Low,0.999021,0.998460,1.000000,0.999165,-0.122823,0.999165,0.999165,0.999165,0.999165,0.999165,...,-0.006646,-0.051339,-0.051339,-0.051339,-0.051339,-0.051339,-0.039536,-0.042448,-0.040352,NaN
Close,0.998199,0.998978,0.999165,1.000000,-0.113115,1.000000,1.000000,1.000000,1.000000,1.000000,...,-0.006068,-0.049754,-0.049754,-0.049754,-0.049754,-0.049754,-0.040237,-0.043190,-0.041101,NaN
Volume,-0.114669,-0.102897,-0.122823,-0.113115,1.000000,-0.113115,-0.113115,-0.113115,-0.113115,-0.113115,...,0.037059,0.327390,0.327390,0.327390,0.327390,0.327390,0.025999,0.025763,0.025329,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F96,-0.046304,-0.044692,-0.051339,-0.049754,0.327390,-0.049754,-0.049754,-0.049754,-0.049754,-0.049754,...,0.012012,1.000000,1.000000,1.000000,1.000000,1.000000,0.035352,0.036484,0.035191,NaN
F97,-0.036349,-0.036955,-0.039536,-0.040237,0.025999,-0.040237,-0.040237,-0.040237,-0.040237,-0.040237,...,-0.021676,0.035352,0.035352,0.035352,0.035352,0.035352,1.000000,0.997443,0.998742,NaN
F98,-0.039283,-0.039891,-0.042448,-0.043190,0.025763,-0.043190,-0.043190,-0.043190,-0.043190,-0.043190,...,-0.020452,0.036484,0.036484,0.036484,0.036484,0.036484,0.997443,1.000000,0.998755,NaN
F99,-0.037199,-0.037827,-0.040352,-0.041101,0.025329,-0.041101,-0.041101,-0.041101,-0.041101,-0.041101,...,-0.021850,0.035191,0.035191,0.035191,0.035191,0.035191,0.998742,0.998755,1.000000,NaN


Notice: The path should be your right file path.

In [35]:
training_data = pd.read_pickle(r'/content/drive/My Drive/training_set.pkl')

I think in this project, features are also kind of hyperparameters. So, I just tune a lot of times.

In [36]:
for stock in range(2000):
    del training_data[stock]["High"]
    del training_data[stock]["Low"]
    del training_data[stock]["Volume"]
    del training_data[stock]["Open"]

    percent_change = []
    for row in range(2201):
        percentage_change = (training_data[stock]['Close'][row+1]-training_data[stock]['Close'][row])/(training_data[stock]['Close'][row])
        percent_change.append(percentage_change)

    percent_change = np.array(percent_change)
    mean = percent_change.mean()
    std = percent_change.std()
    percent_change_norm = (percent_change - mean)/ std
    percent_change = np.append(percent_change,0)
    percent_change_norm = np.append(percent_change_norm,0)

    training_data[stock] = training_data[stock].assign(daily_change = percent_change)
    training_data[stock] = training_data[stock].assign(daily_change_norm = percent_change_norm)

    del training_data[stock]["Close"]

In [37]:
training_data[122]

,daily_change,daily_change_norm
0,-0.001383,-0.129452
1,-0.003190,-0.283521
2,0.004982,0.412967
3,0.003876,0.318687
4,-0.004429,-0.389121
...,...,...
2197,-0.002281,-0.206017
2198,-0.006858,-0.596138
2199,-0.008389,-0.726609
2200,-0.006500,-0.565593


In [38]:
class Stocks_Data(Dataset):
  '''custom dataset'''
  def __init__(self, data_X, data_Y):
    self.X = data_X.to(device)
    self.Y = data_Y.to(device)

  def __len__(self):
    """Returns the size (the number of samples) of the dataset.
    """
    return len(self.Y)
  
  def __getitem__(self, i):
    """Returns the i-th sample and label and applies any transforms defined.

      Args:
        i (int): The index of sample in the data array to retrieve. 
    """  
    label = self.Y[i]
    sample = self.X[i]
    

    return sample, label

Here I read 30 stocks as training data and 10 stocks as validation data. And the window size can also be considered as hyperparameter.

In [39]:
def prepare_training_data_2(training_data, label):
  for stock in range(30):
    for i in range(2201 - 300):
      df = training_data[stock][i:i+300]
      if stock == 0 and i == 0:
        X_train = torch.Tensor(df.values).reshape(1,300,2)
        y_train = torch.Tensor(label[stock][i+300]).reshape(1,3)
      else:
        X_train = torch.cat((X_train, torch.Tensor(df.values).reshape(1,300,2)),0)
        y_train = torch.cat((y_train, torch.Tensor(label[stock][i+300]).reshape(1,3)),0)
      
  
  for stock in range(30,40):
    for i in range(2201 - 300):
      df = training_data[stock][i:i+300]
      if stock == 30 and i == 0:
        X_valid = torch.Tensor(df.values).reshape(1,300,2)
        y_valid = torch.Tensor(label[stock][i+300]).reshape(1,3)
      else:
        X_valid = torch.cat((X_valid, torch.Tensor(df.values).reshape(1,300,2)),0)
        y_valid = torch.cat((y_valid, torch.Tensor(label[stock][i+300]).reshape(1,3)),0)        

  return X_train, y_train, X_valid, y_valid

In [40]:
X_train, y_train, X_valid, y_valid = prepare_training_data_2(training_data, label)

In [41]:
y_train = y_train.to(torch.float32)
y_valid = y_valid.to(torch.float32)

In [42]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

(torch.Size([57030, 300, 2]),
 torch.Size([57030, 3]),
 torch.Size([19010, 300, 2]),
 torch.Size([19010, 3]))

In [43]:
training_dataset = Stocks_Data(X_train, y_train)
valid_dataset = Stocks_Data(X_valid, y_valid)
train_dataloader = DataLoader(training_dataset, batch_size=200)
valid_dataloader = DataLoader(valid_dataset, batch_size=200)

In [44]:
class Model_final_1(nn.Module):
  def __init__(self, input_size, num_class = 3, prob = 0.5):
    super(Model_final_1, self).__init__()
    self.main = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(1024, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(256, 32),
        nn.BatchNorm1d(32),
        nn.ReLU(),
        nn.Dropout(p=prob),
        nn.Linear(32, 3),
    )
  def forward(self, input):
    return self.main(input)

In [45]:
net_1 = Model_final_1(input_size = 600, prob = 0.1).to(device)
learning_rate = 1e-4
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net_1.parameters(), lr=learning_rate)

epochs = 200
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net_1, loss_fn, optimizer)
    if t > 180:
        test_loop(valid_dataloader, net_1, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, net_1, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.715179  [    0/57030]
loss: 0.673062  [20000/57030]
loss: 0.670688  [40000/57030]
Epoch 2
-------------------------------
loss: 0.629687  [    0/57030]
loss: 0.624390  [20000/57030]
loss: 0.627163  [40000/57030]
Epoch 3
-------------------------------
loss: 0.599607  [    0/57030]
loss: 0.605546  [20000/57030]
loss: 0.608632  [40000/57030]
Epoch 4
-------------------------------
loss: 0.575179  [    0/57030]
loss: 0.579692  [20000/57030]
loss: 0.572828  [40000/57030]
Epoch 5
-------------------------------
loss: 0.552700  [    0/57030]
loss: 0.540215  [20000/57030]
loss: 0.529829  [40000/57030]
Epoch 6
-------------------------------
loss: 0.524113  [    0/57030]
loss: 0.532003  [20000/57030]
loss: 0.487448  [40000/57030]
Epoch 7
-------------------------------
loss: 0.482341  [    0/57030]
loss: 0.497471  [20000/57030]
loss: 0.458052  [40000/57030]
Epoch 8
-------------------------------
loss: 0.433389  [    0/57030]
loss: 0.465758  [200

In [46]:
net_2 = Model_final_1(input_size = 600, prob = 0.01).to(device)
learning_rate = 1e-4
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net_2.parameters(), lr=learning_rate)

epochs = 200
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net_2, loss_fn, optimizer)
    if t > 180:
        test_loop(valid_dataloader, net_2, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, net_2, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.733520  [    0/57030]
loss: 0.705345  [20000/57030]
loss: 0.691238  [40000/57030]
Epoch 2
-------------------------------
loss: 0.574489  [    0/57030]
loss: 0.620306  [20000/57030]
loss: 0.617906  [40000/57030]
Epoch 3
-------------------------------
loss: 0.522389  [    0/57030]
loss: 0.554954  [20000/57030]
loss: 0.548779  [40000/57030]
Epoch 4
-------------------------------
loss: 0.470984  [    0/57030]
loss: 0.479612  [20000/57030]
loss: 0.463013  [40000/57030]
Epoch 5
-------------------------------
loss: 0.412621  [    0/57030]
loss: 0.393113  [20000/57030]
loss: 0.354308  [40000/57030]
Epoch 6
-------------------------------
loss: 0.329316  [    0/57030]
loss: 0.291067  [20000/57030]
loss: 0.259389  [40000/57030]
Epoch 7
-------------------------------
loss: 0.243941  [    0/57030]
loss: 0.207285  [20000/57030]
loss: 0.192621  [40000/57030]
Epoch 8
-------------------------------
loss: 0.173256  [    0/57030]
loss: 0.126602  [200

In [47]:
net_3 = Model_final_1(input_size = 600, prob = 0.2).to(device)
learning_rate = 1e-4
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net_3.parameters(), lr=learning_rate)

epochs = 200
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net_3, loss_fn, optimizer)
    if t > 190:
        test_loop(valid_dataloader, net_3, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, net_3, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.740496  [    0/57030]
loss: 0.716516  [20000/57030]
loss: 0.689171  [40000/57030]
Epoch 2
-------------------------------
loss: 0.642313  [    0/57030]
loss: 0.666117  [20000/57030]
loss: 0.650169  [40000/57030]
Epoch 3
-------------------------------
loss: 0.620064  [    0/57030]
loss: 0.646253  [20000/57030]
loss: 0.630753  [40000/57030]
Epoch 4
-------------------------------
loss: 0.607960  [    0/57030]
loss: 0.629329  [20000/57030]
loss: 0.629344  [40000/57030]
Epoch 5
-------------------------------
loss: 0.598828  [    0/57030]
loss: 0.622225  [20000/57030]
loss: 0.612537  [40000/57030]
Epoch 6
-------------------------------
loss: 0.583961  [    0/57030]
loss: 0.599201  [20000/57030]
loss: 0.606728  [40000/57030]
Epoch 7
-------------------------------
loss: 0.578570  [    0/57030]
loss: 0.588291  [20000/57030]
loss: 0.563497  [40000/57030]
Epoch 8
-------------------------------
loss: 0.569823  [    0/57030]
loss: 0.581195  [200

In [48]:
net_4 = Model_final_1(input_size = 600, prob = 0.3).to(device)
learning_rate = 1e-4
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net_4.parameters(), lr=learning_rate)

epochs = 200
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net_4, loss_fn, optimizer)
    if t > 190:
        test_loop(valid_dataloader, net_4, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, net_4, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.702771  [    0/57030]
loss: 0.688351  [20000/57030]
loss: 0.671008  [40000/57030]
Epoch 2
-------------------------------
loss: 0.646465  [    0/57030]
loss: 0.650242  [20000/57030]
loss: 0.650303  [40000/57030]
Epoch 3
-------------------------------
loss: 0.633028  [    0/57030]
loss: 0.632885  [20000/57030]
loss: 0.643816  [40000/57030]
Epoch 4
-------------------------------
loss: 0.630857  [    0/57030]
loss: 0.634551  [20000/57030]
loss: 0.633503  [40000/57030]
Epoch 5
-------------------------------
loss: 0.623971  [    0/57030]
loss: 0.636261  [20000/57030]
loss: 0.626330  [40000/57030]
Epoch 6
-------------------------------
loss: 0.619506  [    0/57030]
loss: 0.631866  [20000/57030]
loss: 0.623874  [40000/57030]
Epoch 7
-------------------------------
loss: 0.609577  [    0/57030]
loss: 0.622183  [20000/57030]
loss: 0.623284  [40000/57030]
Epoch 8
-------------------------------
loss: 0.592246  [    0/57030]
loss: 0.616631  [200

In [49]:
net_5 = Model_final_1(input_size = 600, prob = 0.1).to(device)
learning_rate = 8e-5
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net_5.parameters(), lr=learning_rate)

epochs = 200
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net_5, loss_fn, optimizer)
    if t > 190:
        test_loop(valid_dataloader, net_5, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, net_5, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.710381  [    0/57030]
loss: 0.665664  [20000/57030]
loss: 0.662395  [40000/57030]
Epoch 2
-------------------------------
loss: 0.630865  [    0/57030]
loss: 0.633169  [20000/57030]
loss: 0.635119  [40000/57030]
Epoch 3
-------------------------------
loss: 0.604437  [    0/57030]
loss: 0.618158  [20000/57030]
loss: 0.608697  [40000/57030]
Epoch 4
-------------------------------
loss: 0.581523  [    0/57030]
loss: 0.606859  [20000/57030]
loss: 0.597812  [40000/57030]
Epoch 5
-------------------------------
loss: 0.550168  [    0/57030]
loss: 0.570847  [20000/57030]
loss: 0.587920  [40000/57030]
Epoch 6
-------------------------------
loss: 0.539766  [    0/57030]
loss: 0.554212  [20000/57030]
loss: 0.552897  [40000/57030]
Epoch 7
-------------------------------
loss: 0.508305  [    0/57030]
loss: 0.515105  [20000/57030]
loss: 0.486615  [40000/57030]
Epoch 8
-------------------------------
loss: 0.489874  [    0/57030]
loss: 0.468370  [200

In [50]:
net_6 = Model_final_1(input_size = 600, prob = 0.1).to(device)
learning_rate = 5e-5
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net_6.parameters(), lr=learning_rate)

epochs = 200
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net_6, loss_fn, optimizer)
    if t > 190:
        test_loop(valid_dataloader, net_6, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, net_6, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.695640  [    0/57030]
loss: 0.684442  [20000/57030]
loss: 0.673674  [40000/57030]
Epoch 2
-------------------------------
loss: 0.643279  [    0/57030]
loss: 0.655566  [20000/57030]
loss: 0.646900  [40000/57030]
Epoch 3
-------------------------------
loss: 0.622775  [    0/57030]
loss: 0.639871  [20000/57030]
loss: 0.628405  [40000/57030]
Epoch 4
-------------------------------
loss: 0.608215  [    0/57030]
loss: 0.623833  [20000/57030]
loss: 0.612818  [40000/57030]
Epoch 5
-------------------------------
loss: 0.598166  [    0/57030]
loss: 0.612027  [20000/57030]
loss: 0.611756  [40000/57030]
Epoch 6
-------------------------------
loss: 0.578843  [    0/57030]
loss: 0.592730  [20000/57030]
loss: 0.594067  [40000/57030]
Epoch 7
-------------------------------
loss: 0.557141  [    0/57030]
loss: 0.578840  [20000/57030]
loss: 0.570523  [40000/57030]
Epoch 8
-------------------------------
loss: 0.537042  [    0/57030]
loss: 0.552267  [200

In [51]:
net_7 = Model_final_1(input_size = 600, prob = 0.1).to(device)
learning_rate = 2e-4
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net_7.parameters(), lr=learning_rate)

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net_7, loss_fn, optimizer)
    if t > 90:
        test_loop(valid_dataloader, net_7, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, net_7, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.690088  [    0/57030]
loss: 0.657552  [20000/57030]
loss: 0.647943  [40000/57030]
Epoch 2
-------------------------------
loss: 0.603016  [    0/57030]
loss: 0.622137  [20000/57030]
loss: 0.614113  [40000/57030]
Epoch 3
-------------------------------
loss: 0.566441  [    0/57030]
loss: 0.593681  [20000/57030]
loss: 0.578465  [40000/57030]
Epoch 4
-------------------------------
loss: 0.528137  [    0/57030]
loss: 0.550074  [20000/57030]
loss: 0.525852  [40000/57030]
Epoch 5
-------------------------------
loss: 0.500759  [    0/57030]
loss: 0.483834  [20000/57030]
loss: 0.482121  [40000/57030]
Epoch 6
-------------------------------
loss: 0.453669  [    0/57030]
loss: 0.410120  [20000/57030]
loss: 0.427089  [40000/57030]
Epoch 7
-------------------------------
loss: 0.389563  [    0/57030]
loss: 0.424029  [20000/57030]
loss: 0.396369  [40000/57030]
Epoch 8
-------------------------------
loss: 0.344332  [    0/57030]
loss: 0.331095  [200

In [52]:
net_8 = Model_final_1(input_size = 600, prob = 0.1).to(device)
learning_rate = 5e-5
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net_8.parameters(), lr=learning_rate)

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net_8, loss_fn, optimizer)
    if t > 90:
        test_loop(valid_dataloader, net_8, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, net_8, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.684584  [    0/57030]
loss: 0.670841  [20000/57030]
loss: 0.662866  [40000/57030]
Epoch 2
-------------------------------
loss: 0.637855  [    0/57030]
loss: 0.642022  [20000/57030]
loss: 0.638308  [40000/57030]
Epoch 3
-------------------------------
loss: 0.616093  [    0/57030]
loss: 0.627390  [20000/57030]
loss: 0.619574  [40000/57030]
Epoch 4
-------------------------------
loss: 0.604941  [    0/57030]
loss: 0.611131  [20000/57030]
loss: 0.614232  [40000/57030]
Epoch 5
-------------------------------
loss: 0.593018  [    0/57030]
loss: 0.606576  [20000/57030]
loss: 0.588319  [40000/57030]
Epoch 6
-------------------------------
loss: 0.575770  [    0/57030]
loss: 0.583605  [20000/57030]
loss: 0.571216  [40000/57030]
Epoch 7
-------------------------------
loss: 0.554164  [    0/57030]
loss: 0.578697  [20000/57030]
loss: 0.551500  [40000/57030]
Epoch 8
-------------------------------
loss: 0.534731  [    0/57030]
loss: 0.532380  [200

In [53]:
net_9 = Model_final_1(input_size = 600, prob = 0.2).to(device)
learning_rate = 2e-4
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net_9.parameters(), lr=learning_rate)

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net_9, loss_fn, optimizer)
    if t > 90:
        test_loop(valid_dataloader, net_9, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, net_9, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.742089  [    0/57030]
loss: 0.689995  [20000/57030]
loss: 0.683600  [40000/57030]
Epoch 2
-------------------------------
loss: 0.622405  [    0/57030]
loss: 0.628570  [20000/57030]
loss: 0.649440  [40000/57030]
Epoch 3
-------------------------------
loss: 0.600030  [    0/57030]
loss: 0.608040  [20000/57030]
loss: 0.615854  [40000/57030]
Epoch 4
-------------------------------
loss: 0.586659  [    0/57030]
loss: 0.610317  [20000/57030]
loss: 0.584953  [40000/57030]
Epoch 5
-------------------------------
loss: 0.579553  [    0/57030]
loss: 0.587823  [20000/57030]
loss: 0.572170  [40000/57030]
Epoch 6
-------------------------------
loss: 0.556128  [    0/57030]
loss: 0.535850  [20000/57030]
loss: 0.544496  [40000/57030]
Epoch 7
-------------------------------
loss: 0.532001  [    0/57030]
loss: 0.544611  [20000/57030]
loss: 0.513512  [40000/57030]
Epoch 8
-------------------------------
loss: 0.522349  [    0/57030]
loss: 0.506406  [200

In [54]:
net_X = Model_final_1(input_size = 600, prob = 0.2).to(device)
learning_rate = 5e-5
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net_X.parameters(), lr=learning_rate)

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net_X, loss_fn, optimizer)
    if t > 90:
        test_loop(valid_dataloader, net_X, loss_fn) # validation on validation dataset
        test_loop(train_dataloader, net_X, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.757718  [    0/57030]
loss: 0.735123  [20000/57030]
loss: 0.730174  [40000/57030]
Epoch 2
-------------------------------
loss: 0.698411  [    0/57030]
loss: 0.704564  [20000/57030]
loss: 0.685072  [40000/57030]
Epoch 3
-------------------------------
loss: 0.669562  [    0/57030]
loss: 0.668374  [20000/57030]
loss: 0.665087  [40000/57030]
Epoch 4
-------------------------------
loss: 0.643725  [    0/57030]
loss: 0.654775  [20000/57030]
loss: 0.648514  [40000/57030]
Epoch 5
-------------------------------
loss: 0.630355  [    0/57030]
loss: 0.637370  [20000/57030]
loss: 0.643928  [40000/57030]
Epoch 6
-------------------------------
loss: 0.624739  [    0/57030]
loss: 0.638846  [20000/57030]
loss: 0.634940  [40000/57030]
Epoch 7
-------------------------------
loss: 0.622543  [    0/57030]
loss: 0.634491  [20000/57030]
loss: 0.616648  [40000/57030]
Epoch 8
-------------------------------
loss: 0.617203  [    0/57030]
loss: 0.616741  [200

## Task 5

### Model Ensembling(voting)

In [55]:
def test_loop_voting(dataloader,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_9,model_X):
    size = len(dataloader.dataset)
    correct = 0
    true_positive, pred_positive = 0, 0

    model_1.eval()
    model_2.eval()
    model_3.eval()
    model_4.eval()
    model_5.eval()
    model_6.eval()
    model_7.eval()
    model_8.eval()
    model_9.eval()
    model_X.eval()
    with torch.no_grad():
        for X, y in dataloader:
            a = 0
            b = 0
            c = 0
            pred_1 = model_1(X)
            pred_2 = model_2(X)
            pred_3 = model_3(X)
            pred_4 = model_4(X)
            pred_5 = model_5(X)
            pred_6 = model_6(X)
            pred_7 = model_7(X)
            pred_8 = model_8(X)
            pred_9 = model_9(X)
            pred_X = model_X(X)
            for i in range(len(y)):
                if pred_1[i].argmax().type(torch.float).item()==0:
                    a += 1
                if pred_1[i].argmax().type(torch.float).item()==1:
                    b += 1
                if pred_1[i].argmax().type(torch.float).item()==2:
                    c += 1
                if pred_2[i].argmax().type(torch.float).item()==0:
                    a += 1
                if pred_2[i].argmax().type(torch.float).item()==1:
                    b += 1
                if pred_2[i].argmax().type(torch.float).item()==2:
                    c += 1
                if pred_3[i].argmax().type(torch.float).item()==0:
                    a += 1
                if pred_3[i].argmax().type(torch.float).item()==1:
                    b += 1
                if pred_3[i].argmax().type(torch.float).item()==2:
                    c += 1
                if pred_4[i].argmax().type(torch.float).item()==0:
                    a += 1
                if pred_4[i].argmax().type(torch.float).item()==1:
                    b += 1
                if pred_4[i].argmax().type(torch.float).item()==2:
                    c += 1
                if pred_5[i].argmax().type(torch.float).item()==0:
                    a += 1
                if pred_5[i].argmax().type(torch.float).item()==1:
                    b += 1
                if pred_5[i].argmax().type(torch.float).item()==2:
                    c += 1
                if pred_6[i].argmax().type(torch.float).item()==0:
                    a += 1
                if pred_6[i].argmax().type(torch.float).item()==1:
                    b += 1
                if pred_6[i].argmax().type(torch.float).item()==2:
                    c += 1
                if pred_7[i].argmax().type(torch.float).item()==0:
                    a += 1
                if pred_7[i].argmax().type(torch.float).item()==1:
                    b += 1
                if pred_7[i].argmax().type(torch.float).item()==2:
                    c += 1
                if pred_8[i].argmax().type(torch.float).item()==0:
                    a += 1
                if pred_8[i].argmax().type(torch.float).item()==1:
                    b += 1
                if pred_8[i].argmax().type(torch.float).item()==2:
                    c += 1
                if pred_9[i].argmax().type(torch.float).item()==0:
                    a += 1
                if pred_9[i].argmax().type(torch.float).item()==1:
                    b += 1
                if pred_9[i].argmax().type(torch.float).item()==2:
                    c += 1
                if pred_X[i].argmax().type(torch.float).item()==0:
                    a += 1
                if pred_X[i].argmax().type(torch.float).item()==1:
                    b += 1
                if pred_X[i].argmax().type(torch.float).item()==2:
                    c += 1
                
                if c == max(a,b,c):
                    pred = torch.Tensor([0,0,1])
                elif b == max(a,b,c):
                    pred = torch.Tensor([0,1,0])
                elif a == max(a,b,c):
                    pred = torch.Tensor([1,0,0])
                
                if pred.argmax().type(torch.float).item()>1:
                    pred_positive += 1
                    
                if pred.argmax().type(torch.float).item() == y[i].argmax().type(torch.float).item():
                    correct += 1
                    if pred.argmax().type(torch.float).item()>1:
                        true_positive += 1
                        

    percentage_of_positive_predictions = pred_positive / size
    precision = true_positive / pred_positive
    correct /= size
    print(f"Validation: \n Accuracy: {(100*correct):>0.1f}%, Precision: {(100*precision):>0.1f}%, \
Percentage of positive predictions: {(100*percentage_of_positive_predictions):>0.1f}%")

In [56]:
test_loop_voting(valid_dataloader, net_1,net_2,net_3,net_4,net_5,net_6,net_7,net_8,net_9,net_X) # validation on validation dataset
test_loop_voting(train_dataloader, net_1,net_2,net_3,net_4,net_5,net_6,net_7,net_8,net_9,net_X) # validation on training dataset

Validation: 
 Accuracy: 35.8%, Precision: 34.4%, Percentage of positive predictions: 26.8%
Validation: 
 Accuracy: 41.3%, Precision: 40.2%, Percentage of positive predictions: 28.4%


### Model Ensembling(blending)

In [57]:
class Meta_Model(nn.Module):
  def __init__(self, input_size = 3, num_class = 3):
    super(Meta_Model, self).__init__()
    self.main = nn.Sequential(
        nn.Linear(input_size, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Linear(128, 32),
        nn.BatchNorm1d(32),
        nn.ReLU(),
        nn.Linear(32, 3),
    )
  def forward(self, input):
    return self.main(input)

In [58]:
def train_loop_blending(dataloader, meta_model, model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_9,model_X, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model_1.eval()
    model_2.eval()
    model_3.eval()
    model_4.eval()
    model_5.eval()
    model_6.eval()
    model_7.eval()
    model_8.eval()
    model_9.eval()
    model_X.eval()
    meta_model.train()
    for batch, (X, y) in enumerate(dataloader):   
        # Compute prediction and loss
        pred_1 = model_1(X)
        pred_2 = model_2(X)
        pred_3 = model_3(X)
        pred_4 = model_4(X)
        pred_5 = model_5(X)
        pred_6 = model_6(X)
        pred_7 = model_7(X)
        pred_8 = model_8(X)
        pred_9 = model_9(X)
        pred_X = model_X(X)
        #print(pred_1.shape)
        for i in range(1,11):
            if i == 1:
                pred = meta_model(pred_1).to(torch.float32)
                loss = loss_fn(pred, y)
                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            if i == 2:
                pred = meta_model(pred_2).to(torch.float32)
                loss = loss_fn(pred, y)
                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            if i == 3:
                pred = meta_model(pred_3).to(torch.float32)
                loss = loss_fn(pred, y)
                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            if i == 4:
                pred = meta_model(pred_4).to(torch.float32)
                loss = loss_fn(pred, y)
                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            if i == 5:
                pred = meta_model(pred_5).to(torch.float32)
                loss = loss_fn(pred, y)
                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            if i == 6:
                pred = meta_model(pred_6).to(torch.float32)
                loss = loss_fn(pred, y)
                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            if i == 7:
                pred = meta_model(pred_7).to(torch.float32)
                loss = loss_fn(pred, y)
                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            if i == 8:
                pred = meta_model(pred_8).to(torch.float32)
                loss = loss_fn(pred, y)
                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            if i == 9:
                pred = meta_model(pred_9).to(torch.float32)
                loss = loss_fn(pred, y)
                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            if i == 10:
                pred = meta_model(pred_X).to(torch.float32)
                loss = loss_fn(pred, y)
                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()           

        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [59]:
def prepare_holdout_data(training_data, label):
  for stock in range(500,510):
    for i in range(2201 - 300):
      df = training_data[stock][i:i+300]
      if stock == 500 and i == 0:
        X_hold = torch.Tensor(df.values).reshape(1,300,2)
        y_hold = torch.Tensor(label[stock][i+300]).reshape(1,3)
      else:
        X_hold = torch.cat((X_hold, torch.Tensor(df.values).reshape(1,300,2)),0)
        y_hold = torch.cat((y_hold, torch.Tensor(label[stock][i+300]).reshape(1,3)),0)
             

  return X_hold, y_hold

In [60]:
def test_loop_blending(dataloader, meta_model, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    true_positive, pred_positive = 0, 0
    meta_model.eval()
    model.eval()

    with torch.no_grad():
        for X, y in dataloader:
            
            pred = model(X)
            pred = meta_model(pred)

            test_loss += loss_fn(pred, y).item()
            #correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
            for i in range(len(y)):
                
                if pred[i].argmax().type(torch.float).item()>1:
                    pred_positive += 1
                    
                if pred[i].argmax().type(torch.float).item() == y[i].argmax().type(torch.float).item():
                    correct += 1
                    if pred[i].argmax().type(torch.float).item()>1:
                        true_positive += 1
                        

    percentage_of_positive_predictions = pred_positive / size
    precision = true_positive / pred_positive
    test_loss /= num_batches
    correct /= size
    print(f"Validation: \n Accuracy: {(100*correct):>0.1f}%, Precision: {(100*precision):>0.1f}%, \
Percentage of positive predictions: {(100*percentage_of_positive_predictions):>0.1f}%, Avg loss: {test_loss:>8f}")

In [61]:
X_hold, y_hold = prepare_holdout_data(training_data, label)
hold_dataset = Stocks_Data(X_hold, y_hold)
hold_dataloader = DataLoader(hold_dataset, batch_size=200)

In [62]:
blending = Meta_Model().to(device)
learning_rate = 1e-4
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(blending.parameters(), lr=learning_rate)

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop_blending(hold_dataloader, blending, net_1,net_2,net_3,net_4,net_5,net_6,net_7,net_8,net_9,net_X, loss_fn, optimizer)
    
test_loop_blending(valid_dataloader, blending, net_1, loss_fn) # validation on validation dataset
test_loop_blending(train_dataloader, blending, net_1, loss_fn) # validation on training dataset
print("Done!")

Epoch 1
-------------------------------
loss: 0.710890  [    0/19010]
loss: 0.610422  [10000/19010]
Epoch 2
-------------------------------
loss: 0.639165  [    0/19010]
loss: 0.608062  [10000/19010]
Epoch 3
-------------------------------
loss: 0.637870  [    0/19010]
loss: 0.607496  [10000/19010]
Epoch 4
-------------------------------
loss: 0.636796  [    0/19010]
loss: 0.607477  [10000/19010]
Epoch 5
-------------------------------
loss: 0.636603  [    0/19010]
loss: 0.607593  [10000/19010]
Epoch 6
-------------------------------
loss: 0.636760  [    0/19010]
loss: 0.607560  [10000/19010]
Epoch 7
-------------------------------
loss: 0.636601  [    0/19010]
loss: 0.607677  [10000/19010]
Epoch 8
-------------------------------
loss: 0.636563  [    0/19010]
loss: 0.607718  [10000/19010]
Epoch 9
-------------------------------
loss: 0.636521  [    0/19010]
loss: 0.607799  [10000/19010]
Epoch 10
-------------------------------
loss: 0.636350  [    0/19010]
loss: 0.608007  [10000/19010]

### AdaBoosting

In [63]:
def ada_loop(y_pred, y):
    size = len(y)
    correct = 0
    true_positive, pred_positive = 0, 0

    with torch.no_grad():        
        pred = y

        for i in range(len(y)):
                
            if pred[i].argmax().type(torch.float).item()>1:
               pred_positive += 1
                    
            if pred[i].argmax().type(torch.float).item() == y[i].argmax().type(torch.float).item():
                correct += 1
                if pred[i].argmax().type(torch.float).item()>1:
                    true_positive += 1
                        

    percentage_of_positive_predictions = pred_positive / size
    precision = true_positive / pred_positive
    correct /= size
    print(f"Validation: \n Accuracy: {(100*correct):>0.1f}%, Precision: {(100*precision):>0.1f}%, \
Percentage of positive predictions: {(100*percentage_of_positive_predictions):>0.1f}%")

## Task 6

In task 5, voting and blending do not significantly improved the performance. So, in task 6, I will choose best model from 10 models of task 4.

In [64]:
testing_data_1 = pd.read_pickle(r'/content/drive/My Drive/testing_set1.pkl')
testing_data_2 = pd.read_pickle(r'/content/drive/My Drive/testing_set2.pkl')

In [65]:
len(testing_data_1),len(testing_data_1[0])

(2447, 889)

In [66]:
len(testing_data_2),len(testing_data_2[0])

(705, 2202)

In [67]:
label_1 = []
for stock in range(2447):
  label_1.append([])
for stock in range(2447):
  a = 0
  b = 0
  c = 0
  percent_change = []
  for row in range(888):
    percentage_change = (testing_data_1[stock]['Close'][row+1]-testing_data_1[stock]['Close'][row])/(testing_data_1[stock]['Close'][row])
    percent_change.append(percentage_change)
  
  percent_change_origin = percent_change.copy()
  percent_change.sort()

  
  for row in range(888):
    if percent_change_origin[row] < percent_change[296]:
      label_1[stock].append([1,0,0])
      a += 1
    if percent_change_origin[row] >= percent_change[296] and percent_change_origin[row] < percent_change[592]:
      label_1[stock].append([0,1,0])
      b += 1
    if percent_change_origin[row] >= percent_change[592]:
      label_1[stock].append([0,0,1])
      c += 1

label_1 = np.array(label_1)
a,b,c # for verification

(296, 296, 296)

In [68]:
label_2 = []
for stock in range(705):
  label_2.append([])
for stock in range(705):
  a = 0
  b = 0
  c = 0
  percent_change = []
  for row in range(2201):
    percentage_change = (testing_data_2[stock]['Close'][row+1]-testing_data_2[stock]['Close'][row])/(testing_data_2[stock]['Close'][row])
    percent_change.append(percentage_change)
  
  percent_change_origin = percent_change.copy()
  percent_change.sort()

  
  for row in range(2201):
    if percent_change_origin[row] < percent_change[733]:
      label_2[stock].append([1,0,0])
      a += 1
    if percent_change_origin[row] >= percent_change[733] and percent_change_origin[row] < percent_change[1467]:
      label_2[stock].append([0,1,0])
      b += 1
    if percent_change_origin[row] >= percent_change[1467]:
      label_2[stock].append([0,0,1])
      c += 1

label_2 = np.array(label_2)
a,b,c # for verification

(733, 734, 734)

In [69]:
for stock in range(2447):
    del testing_data_1[stock]["High"]
    del testing_data_1[stock]["Low"]
    del testing_data_1[stock]["Volume"]
    del testing_data_1[stock]["Open"]

    percent_change = []
    for row in range(888):
        percentage_change = (testing_data_1[stock]['Close'][row+1]-testing_data_1[stock]['Close'][row])/(testing_data_1[stock]['Close'][row])
        percent_change.append(percentage_change)
    
    percent_change = np.array(percent_change)
    mean = percent_change.mean()
    std = percent_change.std()
    percent_change_norm = (percent_change - mean)/ std
    percent_change = np.append(percent_change,0)
    percent_change_norm = np.append(percent_change_norm,0)
    
    testing_data_1[stock] = testing_data_1[stock].assign(daily_change = percent_change)
    testing_data_1[stock] = testing_data_1[stock].assign(daily_change_norm = percent_change_norm)

    del testing_data_1[stock]["Close"]

In [70]:
testing_data_1[2400]

,daily_change,daily_change_norm
0,-0.000993,-0.073298
1,0.008942,0.751398
2,-0.016297,-1.343802
3,-0.011963,-0.983957
4,0.020263,1.691236
...,...,...
884,0.000290,0.033157
885,-0.008983,-0.736598
886,-0.000292,-0.015179
887,0.002632,0.227620


In [71]:
for stock in range(705):
    del testing_data_2[stock]["High"]
    del testing_data_2[stock]["Low"]
    del testing_data_2[stock]["Volume"]
    del testing_data_2[stock]["Open"]

    percent_change = []
    for row in range(2201):
        percentage_change = (testing_data_2[stock]['Close'][row+1]-testing_data_2[stock]['Close'][row])/(testing_data_2[stock]['Close'][row])
        percent_change.append(percentage_change)
    
    percent_change = np.array(percent_change)
    mean = percent_change.mean()
    std = percent_change.std()
    percent_change_norm = (percent_change - mean)/ std
    percent_change = np.append(percent_change,0)
    percent_change_norm = np.append(percent_change_norm,0)
    
    testing_data_2[stock] = testing_data_2[stock].assign(daily_change = percent_change)
    testing_data_2[stock] = testing_data_2[stock].assign(daily_change_norm = percent_change_norm)

    del testing_data_2[stock]["Close"]

In [72]:
def prepare_testing_data_1(testing_data_1, label):
  for stock in range(50):
    for i in range(888 - 300):
      df = testing_data_1[stock][i:i+300]
      if stock == 0 and i == 0:
        X_test_1 = torch.Tensor(df.values).reshape(1,300,2)
        y_test_1 = torch.Tensor(label[stock][i+300]).reshape(1,3)
      else:
        X_test_1 = torch.cat((X_test_1, torch.Tensor(df.values).reshape(1,300,2)),0)
        y_test_1 = torch.cat((y_test_1, torch.Tensor(label[stock][i+300]).reshape(1,3)),0)
       

  return X_test_1, y_test_1

In [73]:
def prepare_testing_data_2(testing_data_2, label):
  for stock in range(20):
    for i in range(2201 - 300):
      df = testing_data_2[stock][i:i+300]
      if stock == 0 and i == 0:
        X_test_2 = torch.Tensor(df.values).reshape(1,300,2)
        y_test_2 = torch.Tensor(label[stock][i+300]).reshape(1,3)
      else:
        X_test_2 = torch.cat((X_test_2, torch.Tensor(df.values).reshape(1,300,2)),0)
        y_test_2 = torch.cat((y_test_2, torch.Tensor(label[stock][i+300]).reshape(1,3)),0)
           

  return X_test_2, y_test_2

In [74]:
X_test_1, y_test_1 = prepare_testing_data_1(testing_data_1, label_1)

In [75]:
X_test_2, y_test_2 = prepare_testing_data_2(testing_data_2, label_2)

In [76]:
X_test_1.shape, y_test_1.shape

(torch.Size([29400, 300, 2]), torch.Size([29400, 3]))

In [77]:
X_test_2.shape, y_test_2.shape

(torch.Size([38020, 300, 2]), torch.Size([38020, 3]))

In [78]:
testing_dataset_1 = Stocks_Data(X_test_1, y_test_1)
testing_dataloader_1 = DataLoader(testing_dataset_1, batch_size=200)

In [79]:
testing_dataset_2 = Stocks_Data(X_test_2, y_test_2)
testing_dataloader_2 = DataLoader(testing_dataset_2, batch_size=200)

In [113]:
test_loop(testing_dataloader_1, net_1, loss_fn)

Validation: 
 Accuracy: 36.4%, Precision: 34.4%, Percentage of positive predictions: 27.3%, Avg loss: 1.953579


In [114]:
test_loop(testing_dataloader_2, net_1, loss_fn)

Validation: 
 Accuracy: 44.2%, Precision: 46.8%, Percentage of positive predictions: 27.9%, Avg loss: 1.757232


## Private testing set

### If you want to test my model on private testing set, here what you should do.

Frist, load your private testing set. Below is an example.

In [82]:
testing_data_private = pd.read_pickle(r'/content/drive/My Drive/testing_set3.pkl')

In [83]:
len(testing_data_private), len(testing_data_private[200])

(4598, 483)

Second, adding labels for the testing set. You should revise some setting refering to the notes.

In [84]:
label_3 = []
for stock in range(4598): # Notice: Here 4598 is the length of testing set which represents that it has 4598 stocks
  label_3.append([])
for stock in range(4598): # Notice: Here 4598 is the length of testing set which represents that it has 4598 stocks
  a = 0
  b = 0
  c = 0
  percent_change = []
  for row in range(482): 
    percentage_change = (testing_data_private[stock]['Close'][row+1]-testing_data_private[stock]['Close'][row])/(testing_data_private[stock]['Close'][row])
    percent_change.append(percentage_change)
  
  percent_change_origin = percent_change.copy()
  percent_change.sort()

  
  for row in range(482): 
    if percent_change_origin[row] < percent_change[161]: 
      label_3[stock].append([1,0,0])
      a += 1
    if percent_change_origin[row] >= percent_change[161] and percent_change_origin[row] < percent_change[322]:
      label_3[stock].append([0,1,0])
      b += 1
    if percent_change_origin[row] >= percent_change[322]:
      label_3[stock].append([0,0,1])
      c += 1

label_3 = np.array(label_3)
a,b,c # for verification

(161, 161, 160)

Third, feature engineering.

In [85]:
for stock in range(4598):
    del testing_data_private[stock]["High"]
    del testing_data_private[stock]["Low"]
    del testing_data_private[stock]["Volume"]
    del testing_data_private[stock]["Open"]

    percent_change = []
    for row in range(482):
        percentage_change = (testing_data_private[stock]['Close'][row+1]-testing_data_private[stock]['Close'][row])/(testing_data_private[stock]['Close'][row])
        percent_change.append(percentage_change)
    
    percent_change = np.array(percent_change)
    mean = percent_change.mean()
    std = percent_change.std()
    percent_change_norm = (percent_change - mean)/ std
    percent_change = np.append(percent_change,0)
    percent_change_norm = np.append(percent_change_norm,0)
    
    testing_data_private[stock] = testing_data_private[stock].assign(daily_change = percent_change)
    testing_data_private[stock] = testing_data_private[stock].assign(daily_change_norm = percent_change_norm)

    del testing_data_private[stock]["Close"]

Fourth, tranform list of dataframe to torch tensor.

In [86]:
def prepare_testing_data_private(testing_data_private, label):
  for stock in range(100): 
    for i in range(482 - 300): # Notice: I use 300 days to predict next label
      df = testing_data_private[stock][i:i+300]
      if stock == 0 and i == 0:
        X_test_3 = torch.Tensor(df.values).reshape(1,300,2)
        y_test_3 = torch.Tensor(label[stock][i+300]).reshape(1,3)
      else:
        X_test_3 = torch.cat((X_test_3, torch.Tensor(df.values).reshape(1,300,2)),0)
        y_test_3 = torch.cat((y_test_3, torch.Tensor(label[stock][i+300]).reshape(1,3)),0)
       

  return X_test_3, y_test_3

In [87]:
X_test_private, y_test_private = prepare_testing_data_private(testing_data_private, label_3)

Torch tensor to dataset, dataset to dataloader.

In [88]:
testing_dataset_private = Stocks_Data(X_test_private, y_test_private)
testing_dataloader_private = DataLoader(testing_dataset_private, batch_size=200)

Finally, dataloader given to my best model to see the three metrics

In [115]:
test_loop(testing_dataloader_private, net_1, loss_fn)

Validation: 
 Accuracy: 36.0%, Precision: 34.7%, Percentage of positive predictions: 24.5%, Avg loss: 2.028909


# Summary: My model only performs well on validation set and testset2. And the performance is consistent on these two sets. However, on testset1 and testset2 my model has poor performances.

# By the way, if there existing useful knowledge or pattern to be mined between different stocks is a question. I think it is not reasonable enough to do data mining of bundles of stocks and use the mining results to predict another bundles of stocks. What's more, the data is from real world and in real world, nobody uses such method to predict stock prices which should be used to decide trading policy.